In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [6]:
import numpy as np
import igraph as ig
import src.analysis.modularity_vitality as mv_v2
import src.analysis.mod_deltas as mv_v1 

In [7]:
def calcErrors(g, part, mvs):
    errors = []
    membership = part.membership
    for i in range(g.vcount()):
        h = g.copy()
        h.delete_vertices(i)
        h_membership = membership.copy()
        h_membership.pop(i)
        if g.is_weighted():
            h_part = ig.VertexClustering(h, membership=h_membership,
                                        modularity_params={'weights':'weight'})
        else:
            h_part = ig.VertexClustering(h, membership=h_membership)
        true_vitality = part.modularity - h_part.modularity
        error = true_vitality - mvs[i]
        errors.append(error)
    return errors

In [8]:
n = 5000
p = 0.001

In [15]:
# standard graph
g = ig.Graph.Erdos_Renyi(n=n, p=p, loops=False)
part = g.community_leiden(objective_function='modularity')

mvs1 = mv_v1.mod_deltas(g, part)
mvs1 = list(-1 * np.array(mvs1))
errors = calcErrors(g, part, mvs1)
print('v1:')
print(f'min error: {np.min(errors)}, max error:{np.max(errors)}')

mvs2 = mv_v2.modularity_vitality(g, part)
errors = calcErrors(g, part, mvs2)
print('v2:')
print(f'min error: {np.min(errors)}, max error:{np.max(errors)}')

v1:
min error: -2.7755575615628914e-16, max error:2.220446049250313e-16
v2:
min error: -2.7755575615628914e-16, max error:2.220446049250313e-16


In [16]:
np.array(mvs1) - np.array(mvs2)

array([0., 0., 0., ..., 0., 0., 0.])

In [33]:
# graph w/ loops
g = ig.Graph.Erdos_Renyi(n=n, p=p, loops=True)
part = g.community_leiden(objective_function='modularity')

mvs = mv_v1.mod_deltas(g, part)
errors = calcErrors(g, part, mvs)
print('v1:')
print(f'min error: {np.min(errors)}, max error:{np.max(errors)}')

mvs = mv_v2.modularity_vitality(g, part)
errors = calcErrors(g, part, mvs)
print('v2:')
print(f'min error: {np.min(errors)}, max error:{np.max(errors)}')

v1:
min error: -3.3306690738754696e-16, max error:3.3306690738754696e-16
v2:
min error: -3.3306690738754696e-16, max error:3.3306690738754696e-16


In [46]:
# standard weighted graph
g = ig.Graph.Erdos_Renyi(n=n, p=p, loops=False)
weights = 100 * np.random.rand(g.ecount()).tolist()
g.es['weight'] = weights
part = g.community_leiden(objective_function='modularity',
                         weights='weight')

mvs = mv_v1.mod_deltas(g, part)
errors = calcErrors(g, part, mvs)
print('v1:')
print(f'min error: {np.min(errors)}, max error:{np.max(errors)}')

mvs = mv_v2.modularity_vitality(g, part)
errors = calcErrors(g, part, mvs)
print('v2:')
print(f'min error: {np.min(errors)}, max error:{np.max(errors)}')

v1:
min error: 2.4424906541753444e-15, max error:3.774758283725532e-15
v2:
min error: 2.4424906541753444e-15, max error:3.774758283725532e-15


In [47]:
# weighted graph w/ loops
g = ig.Graph.Erdos_Renyi(n=n, p=p, loops=True)
weights = 100 * np.random.rand(g.ecount()).tolist()
g.es['weight'] = weights
part = g.community_leiden(objective_function='modularity',
                         weights='weight')

mvs = mv_v1.mod_deltas(g, part)
errors = calcErrors(g, part, mvs)
print('v1:')
print(f'min error: {np.min(errors)}, max error:{np.max(errors)}')

mvs = mv_v2.modularity_vitality(g, part)
errors = calcErrors(g, part, mvs)
print('v2:')
print(f'min error: {np.min(errors)}, max error:{np.max(errors)}')

v1:
min error: -1.5543122344752192e-15, max error:-1.1102230246251565e-16
v2:
min error: -1.5543122344752192e-15, max error:-1.1102230246251565e-16


In [36]:
%%timeit
part.recalculate_modularity()

445 µs ± 30.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [37]:
%%timeit
mv_v1.mod_deltas(g, part)

33.1 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
%%timeit
mv_v2.modularity_vitality(g, part)

28.8 ms ± 941 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
print(f'speedup: {.445 / (33.1 /g.vcount())}x')

speedup: 67.22054380664653x


In [40]:
print(f'speedup: {.445 / (28.8 /g.vcount())}x')

speedup: 77.25694444444444x


In [41]:
# n = 500000
# p = 1e-5
# g = ig.Graph.Erdos_Renyi(n=n, p=p, loops=False)
# part = g.community_leiden(objective_function='modularity')

In [42]:
g = ig.Graph.Read_Pickle('election_day_lc.pkl')
part = g.community_leiden(objective_function='modularity',
                         weights='weight')

In [43]:
g.vcount(), g.is_weighted(), g.ecount()

(267278, True, 2142353)

In [44]:
test1 = mv_v1.mod_deltas(g, part)
test2 = mv_v2.modularity_vitality(g, part)

In [45]:
np.abs(np.array(test1) - np.array(test2)).max()

0.0

In [24]:
%%timeit
test1 = mv_v1.mod_deltas(g, part)

4.97 s ± 181 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
test2 = mv_v2.modularity_vitality(g, part)

4.57 s ± 70.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
